<a href="https://colab.research.google.com/github/specfazhou/coding_for_transformer/blob/main/final_results_show.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm
!python -m spacy download it_core_news_sm

     |████████████████████████████████| 12.0 MB 13.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 14.9 MB 13.8 MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-py3-none-any.whl size=14907055 sha256=de0ccecc4984196294bc4e5f3357ab2882c2acc7bada099c9bc08ff8dd500c4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-lnudjwzc/wheels/00/66/69/cb6c921610087d2cab339062345098e30a5ceb665360e7b32a
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
     |████████████████████████████████| 14.5 MB 13.6 MB/s 
  Created wheel for it-core-news-sm: filename=it_core_news_sm-2.2.5-py3-none-any.whl size=14471129 sha256=4204860319c33ffd4c41b8b0f5ab63363e1b8c47d44da73c3e4e0cc852208dcb
  Stored in directory: /tmp/pip-ephem-wheel-cache-16om3ar0/wheels/87/88/46/36fd0cabbebd89b2ee247bf113c1ca4f2cb184f8b7a675

In [13]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
        



In [2]:
transformer_en_de = Seq2SeqTransformer(3, 3, 256, 4, 22125, 37478, 512)
transformer_en_de.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/transformer_en_de.ipynb"))

transformer_de_it = Seq2SeqTransformer(3, 3, 256, 4, 37768, 32717, 512)
transformer_de_it.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/transformer_de_it.ipynb"))

transformer_it_en = Seq2SeqTransformer(3, 3, 256, 4, 32722, 22203, 512)
transformer_it_en.load_state_dict(torch.load("./drive/MyDrive/Colab Notebooks/transformer_it_en.ipynb"))

<All keys matched successfully>

In [9]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import IWSLT2017
from typing import Iterable, List

new_IWSLT2017_en_de = []
new_IWSLT2017_de_it = []
new_IWSLT2017_it_en = []

i = 0
j = 0
k = 0
for data in IWSLT2017(split='train', language_pair=('en','de')):
    if i < 25000:
      new_IWSLT2017_en_de.append(data)
    else:
      break
    i = i + 1

for data in IWSLT2017(split='train', language_pair=('de', 'it')):
    if j < 25000:
      new_IWSLT2017_de_it.append(data)
    else:
      break
    j = j + 1

for data in IWSLT2017(split='train', language_pair=('it', 'en')):
    if k < 25000:
      new_IWSLT2017_it_en.append(data)
    else:
      break
    k = k + 1


In [10]:
a1 =""
a2 = ""
a3 = ""
b1 = ""
b2 = ""
b3 = ""

In [16]:
transformer_en_de = transformer_en_de.to(DEVICE)
transformer_de_it = transformer_de_it.to(DEVICE)
transformer_it_en = transformer_it_en.to(DEVICE)

In [21]:



for suck in [0,1,2]:
  new_IWSLT2017 = []
  token_transform = {}
  vocab_transform = {}

  if suck == 0:
    SRC_LANGUAGE = 'en'
    TGT_LANGUAGE = 'de'
    token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
    token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
    new_IWSLT2017 = new_IWSLT2017_en_de

  if suck == 1:
    SRC_LANGUAGE = 'de'
    TGT_LANGUAGE = 'it'
    token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
    token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='it_core_news_sm')
    new_IWSLT2017 = new_IWSLT2017_de_it

  if suck == 2:
    SRC_LANGUAGE = 'it'
    TGT_LANGUAGE = 'en'
    token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='it_core_news_sm')
    token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
    new_IWSLT2017 = new_IWSLT2017_it_en

  def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

# Define special symbols and indices
  UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
  special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

  for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
      train_iter = new_IWSLT2017
    # Create torchtext's Vocab object
      vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set UNK_IDX as the default index. This index is returned when the token is not found.
# If not set, it throws RuntimeError when the queried token is not found in the Vocabulary.
  for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)





  from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
  def sequential_transforms(*transforms):
      def func(txt_input):
          for transform in transforms:
              txt_input = transform(txt_input)
          return txt_input
      return func

# function to add BOS/EOS and create tensor for input sequence indices
  def tensor_transform(token_ids: List[int]):
      return torch.cat((torch.tensor([BOS_IDX]),
                        torch.tensor(token_ids),
                        torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
  text_transform = {}
  for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
      text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                                 vocab_transform[ln], #Numericalization
                                                 tensor_transform) # Add BOS/EOS and create tensor

  def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


  def create_mask(src, tgt):
      src_seq_len = src.shape[0]
      tgt_seq_len = tgt.shape[0]

      tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
      src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

      src_padding_mask = (src == PAD_IDX).transpose(0, 1)
      tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
      return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask



# function to generate output sequence using greedy algorithm
  def greedy_decode(model, src, src_mask, max_len, start_symbol):
      src = src.to(DEVICE)
      src_mask = src_mask.to(DEVICE)

      memory = model.encode(src, src_mask)
      ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
      for i in range(max_len-1):
          memory = memory.to(DEVICE)
          tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                      .type(torch.bool)).to(DEVICE)
          out = model.decode(ys, memory, tgt_mask)
          out = out.transpose(0, 1)
          prob = model.generator(out[:, -1])
          _, next_word = torch.max(prob, dim=1)
          next_word = next_word.item()

          ys = torch.cat([ys,
                          torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
          if next_word == EOS_IDX:
              break
      return ys


# actual function to translate input sentence into target language
  def translate(model: torch.nn.Module, src_sentence: str):
      model.eval()
      src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
      num_tokens = src.shape[0]
      src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
      tgt_tokens = greedy_decode(
          model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
      return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

  if suck == 0:
    a1 = translate(transformer_en_de, "deep learning is so hard, and I can only use colab with limited computing resource to train my models" )
    a2 = translate(transformer_en_de, "so, I do not calculate loss function on validate and test datas, and my results are not great")
    a3 = translate(transformer_en_de, "hope you will not be harsh on grading")
    print("deep learning is too hard, and I can only use colab with limited computing resource to train my models")
    print("so, I do not calculate loss function on validate and test datas, and my results are not great")
    print("hope you will not be harsh on grading")
    print(a1)
    print(a2)
    print(a3)
  
  if suck == 1:
    b1 = translate(transformer_de_it, a1)
    b2 = translate(transformer_de_it, a2)
    b3 = translate(transformer_de_it, a3)
    print(b1)
    print(b2)
    print(b3)

  if suck == 2:
    print(translate(transformer_it_en, b1))
    print(translate(transformer_it_en, b2))
    print(translate(transformer_it_en, b3))



deep learning is too hard, and I can only use colab with limited computing resource to train my models
so, I do not calculate loss function on validate and test datas, and my results are not great
hope you will not be harsh on grading
 Es ist so so so , und ich kann nur nur mit den ganzen ganzen ganzen ganzen ganzen Weg zu machen . 
 Ich denke nicht , dass die ganze Art und mein System nicht mehr und meine Arbeit nicht sehr gut sind . 
 Die Leute werden nicht auf den gleichen Art 
 E ' molto importante , e ho fatto solo per fare in modo in cui abbiamo fatto . 
 E non ho fatto che la mia vita non è la mia vita e non è la mia vita . 
 E il mondo non si può essere in cui le persone non si può essere
 It 's very important , and I 've been just just just to do it in the way we have to do . 
 It 's not I 'm not my life . 
 It 's the world can not be in the world that people can not be not to be . 
